In [11]:
import pandas as pd
import psycopg2
import io

In [50]:
sp = pd.read_csv('amostraSP.csv')
sp.head()

,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,number,street,city,district,postcode,latGold,lonGold
0,0,212,212,212,45,JANE VANINE CAPOZI,SAO PAULO,PARELHEIROS,NaN,-23.868805,-46.678967
1,1,7364,7364,7364,10,DOS LIRIOS,SAO PAULO,SAO RAFAEL,NaN,-23.638449,-46.447187
2,2,10718,10718,10718,81,ANTONIO XAVIER,SAO PAULO,JARDIM HELENA,NaN,-23.475623,-46.427649
3,3,8118,8118,8118,259,RUA RIBEIRO JUNQUEIRA,SAO PAULO,CACHOEIRINHA,NaN,-23.479475,-46.653269
4,4,474,474,474,738,PEROBEIRAS,SAO PAULO,CAPAO REDONDO,NaN,-23.693233,-46.795505


In [51]:
amostraP = sp.fillna( value={'number':'REMOVER','street':	'REMOVER','city':'REMOVER', 'postcode':''})
numero = sp['number'].to_list()
street = sp['street'].to_list()
city = sp['city'].to_list()
district = sp['district'].to_list()
postcode = sp['postcode'].to_list()



In [19]:
def get_max_request_id(cur):
    
    query = """ SELECT MAX(request_id)
                FROM "Request"
                    """
    cur.execute(query)
    maior = cur.fetchall()
    return maior[0][0]

In [20]:
database_crawler = "SGM"
user_crawler = "terralab"
host_crawler = "34.23.173.147"
password_crawler = "terralab0705"


conn_crawler = psycopg2.connect(database=database_crawler,
                  user=user_crawler,
                  host=host_crawler,
                  password=password_crawler)

cur_crawler = conn_crawler.cursor()

In [21]:
request_id = get_max_request_id(cur_crawler)
request_id

2937599

In [56]:
type(sp['street'][0])

str

In [57]:
query_insercao = """
    INSERT INTO "Request" (
        request_id,
        full_address,
        number,
        public_place,
        district,
        city,
        zip_code,
        state
    )
    VALUES """
for i in range(0, len(numero)):
    if numero[i] == 'REMOVER' or street[i] == 'REMOVER' or city[i] == 'REMOVER':
        continue
    else:
        request_id += 1
        if isinstance(street[i], float):
            street_str = str(street[i])
        else:
            street_str = street[i].replace("'", "''")
        
        full_address = 'SP' + ', ' + street_str + ', ' + str( numero[i]) + ', ' + 'SAO PAULO' 
        query_insercao += """(
            {},
            '{}',
            '{}',
            '{}',
            '{}',
            '{}',
            '{}',
            '{}'),\n""".format(request_id, full_address, 
                                numero[i], street_str,
                                '',
                                city[i].replace("'","''"),
                                '', 'SP')
    

query_insercao =query_insercao[:-2] +';'

In [58]:
print(query_insercao)


    INSERT INTO "Request" (
        request_id,
        full_address,
        number,
        public_place,
        district,
        city,
        zip_code,
        state
    )
    VALUES (
            2932600,
            'SP, JANE VANINE CAPOZI,         45, SAO PAULO',
            '        45',
            'JANE VANINE CAPOZI',
            '',
            'SAO PAULO',
            '',
            'SP'),
(
            2932601,
            'SP, DOS LIRIOS, 10, SAO PAULO',
            '10',
            'DOS LIRIOS',
            '',
            'SAO PAULO',
            '',
            'SP'),
(
            2932602,
            'SP, ANTONIO XAVIER, 81, SAO PAULO',
            '81',
            'ANTONIO XAVIER',
            '',
            'SAO PAULO',
            '',
            'SP'),
(
            2932603,
            'SP, RUA RIBEIRO JUNQUEIRA, 259, SAO PAULO',
            '259',
            'RUA RIBEIRO JUNQUEIRA',
            '',
            'SAO PAULO',
            '',
            '

In [59]:
cur_crawler.execute(query_insercao)
print(request_id)
request_id = get_max_request_id(cur_crawler)
print(request_id)

2937599
2937599


In [60]:
cur_crawler.close()
conn_crawler.commit()